<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/analysis/XGBoost%20for%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io

In [2]:
from google.colab import files

uploaded = files.upload()

Saving processed_training.csv to processed_training.csv


In [3]:
print(uploaded.keys())

dict_keys(['processed_training.csv'])


In [4]:
train_data = pd.read_csv(io.BytesIO(uploaded['processed_training.csv']), sep = ',')

cols_to_keep = ['label', 'sentence', 'tokenized_sentence', 'cleaned_tokens']
train_data = train_data.loc[:, cols_to_keep]
train_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이제 슬슬 자리를 잡았다 싶었는데 아내가 세상을 떠났어.,"['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을...","['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을..."
1,0,내가 돈을 펑펑 쓰고 다닌다면서 남들이 뭐라고 하더라.,"['내가', '돈을', '펑', '##펑', '쓰고', '다닌', '##다면', '...","['내가', '돈을', '펑', '쓰고', '다닌', '남들이', '뭐라고', '하']"
2,0,아무 일도 아닌 것에 스트레스를 받는 나 자신이 너무 혐오스러워.,"['아무', '일도', '아닌', '것에', '스트레스를', '받는', '나', '...","['아무', '일도', '아닌', '것에', '스트레스를', '받는', '나', '..."
3,0,취업하기가 너무 힘들어.,"['취업', '##하기가', '너무', '힘들어', '.']","['취업', '너무', '힘들어']"
4,0,정년퇴직을 하고 나니 노후준비를 안 한 것이 후회가 돼.,"['정', '##년', '##퇴', '##직을', '하고', '나니', '노후', ...","['정', '하고', '나니', '노후', '안', '한', '것이', '후회', ..."


In [5]:
train_data.shape

(24504, 4)

In [6]:
uploaded = files.upload()

Saving processed_test.csv to processed_test.csv


In [7]:
print(uploaded.keys())

dict_keys(['processed_test.csv'])


In [8]:
test_data = pd.read_csv(io.BytesIO(uploaded['processed_test.csv']), sep = ',')

test_data = test_data.loc[:, cols_to_keep]
test_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,"['이번', '프로젝트', '##에서', '발표를', '하는데', '내가', '실수...","['이번', '프로젝트', '발표를', '하는데', '내가', '실수', '바람에'..."
1,0,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,"['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔...","['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔..."
2,0,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,"['상', '##사가', '너무', '무섭게', '생겨서', '친', '##해지는'...","['상', '너무', '무섭게', '생겨서', '친', '게', '너무', '두려워']"
3,0,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,"['이번에', '힘들게', '들어간', '첫', '직장', '##이거', '##든'...","['이번에', '힘들게', '들어간', '첫', '직장', '첫', '직장', '그..."
4,0,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,"['직장', '##에서', '동료', '##들이랑', '관계가', '안', '좋아질...","['직장', '동료', '관계가', '안', '좋아질', '봐', '걱정']"


In [9]:
test_data.shape

(6640, 4)

# Data Preparation

In [10]:
import ast
import torch
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

# Convert the string representation of the list to an actual list
train_data['cleaned_tokens'] = train_data['cleaned_tokens'].apply(ast.literal_eval)
test_data['cleaned_tokens'] = test_data['cleaned_tokens'].apply(ast.literal_eval)

# Flatten the list of tokens
all_tokens = train_data['cleaned_tokens'].explode()

# Build a vocabulary using Counter
vocab = Counter(all_tokens)

# Create word-to-index and index-to-word dictionaries
word_to_index = {word: index + 1 for index, (word, _) in enumerate(vocab.most_common())}
index_to_word = {index: word for word, index in word_to_index.items()}

# Add a special token for padding with index 0
word_to_index[''] = 0
index_to_word[0] = ''

# Add a special token for out-of-vocabulary words with index len(vocab) + 1
word_to_index[''] = len(vocab) + 1
index_to_word[len(vocab) + 1] = ''

# Encode tokens using the word-to-index dictionary, replace out-of-vocabulary words with ''
train_data['encoded_tokens'] = train_data['cleaned_tokens'].apply(lambda tokens: [word_to_index.get(token, word_to_index['']) for token in tokens])

# Pad sequences to a specific length (adjust maxlen as needed)
maxlen = train_data['encoded_tokens'].apply(len).max()
padded_tokens = pad_sequence([torch.tensor(tokens) for tokens in train_data['encoded_tokens']], batch_first=True, padding_value=word_to_index['']).tolist()
train_data['padded_tokens'] = [list(tokens) for tokens in padded_tokens]

In [11]:
train_data.iloc[:,-2:].head()

,encoded_tokens,padded_tokens
0,"[17, 2627, 1306, 1695, 1033, 108, 842, 1456]","[17, 2627, 1306, 1695, 1033, 108, 842, 1456, 1..."
1,"[1, 92, 3587, 1396, 1740, 3218, 444, 249]","[1, 92, 3587, 1396, 1740, 3218, 444, 249, 1137..."
2,"[316, 690, 565, 929, 1019, 622, 8, 227, 3, 1962]","[316, 690, 565, 929, 1019, 622, 8, 227, 3, 196..."
3,"[79, 3, 82]","[79, 3, 82, 11373, 11373, 11373, 11373, 11373,..."
4,"[433, 28, 735, 145, 12, 25, 121, 236, 102]","[433, 28, 735, 145, 12, 25, 121, 236, 102, 113..."


In [12]:
print("Length of the Vocabulary is: ", len(vocab))

Length of the Vocabulary is:  11372


In [13]:
train_data['padded_tokens'].apply(len).unique()

array([38])

In [14]:
# Example: Reconstruct tokens for the first row
example_row_index = 0
reconstructed_tokens = [index_to_word[index] for index in train_data.loc[example_row_index, 'padded_tokens'] if index != 0]

# Display the reconstructed tokens
print(reconstructed_tokens)

['이제', '슬슬', '자리를', '잡았다', '싶었는데', '아내가', '세상을', '떠났', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [17]:
# Encode test tokens using the word-to-index dictionary, replace out-of-vocabulary words with ''
test_data['encoded_tokens'] = test_data['cleaned_tokens'].apply(lambda tokens: [word_to_index.get(token, word_to_index['']) for token in tokens])

# Pad sequences to a specific length (adjust maxlen as needed)
maxlen = test_data['encoded_tokens'].apply(len).max()
padded_tokens = pad_sequence([torch.tensor(tokens) for tokens in test_data['encoded_tokens']], batch_first=True, padding_value=word_to_index['']).tolist()
test_data['padded_tokens'] = [list(tokens) for tokens in padded_tokens]

# Build XGBoost Model

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Extract features (padded_tokens) and labels
features = train_data['padded_tokens'].tolist()
labels = train_data['label']

# Flatten the lists in the 'padded_tokens' column
flattened_features = pd.DataFrame(features)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(flattened_features, labels, test_size=0.2, random_state=42)

# Define XGBoost classifier
model = xgb.XGBClassifier()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7890226484390941
